In [1]:
import fitz  # PyMuPDF
import re
import pandas as pd
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
import os

In [2]:
def read_pdf(file_path):
    text = ""
    with fitz.open(file_path) as pdf_document:
        for page_number in range(pdf_document.page_count):
            page = pdf_document[page_number]
            text += page.get_text()

    return text 

In [3]:
folder_path = "/home/skmlab1/data/data/QuangMinh/Learning/Clone/RAGBOOK/note/train_book"
file_names = []
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        file_names.append(file_path)
file_names

['/home/skmlab1/data/data/QuangMinh/Learning/Clone/RAGBOOK/note/train_book/10550-loan-the-anh-hung-tap-1-thuviensach.vn.pdf',
 '/home/skmlab1/data/data/QuangMinh/Learning/Clone/RAGBOOK/note/train_book/11441-nhung-chang-trai-huyen-thoai-tap-1-thuviensach.vn.pdf',
 '/home/skmlab1/data/data/QuangMinh/Learning/Clone/RAGBOOK/note/train_book/12844-bi-mat-cua-nhung-bi-mat-tap-1-bai-noi-ve-bi-mat-cua-hoa-vang-thuviensach.vn.pdf',
 '/home/skmlab1/data/data/QuangMinh/Learning/Clone/RAGBOOK/note/train_book/13270-vong-tron-da-thieng-tap-1-thuviensach.vn.pdf',
 '/home/skmlab1/data/data/QuangMinh/Learning/Clone/RAGBOOK/note/train_book/13354-world-of-warcraft-chronicle-quyen-1-thuviensach.vn.pdf',
 '/home/skmlab1/data/data/QuangMinh/Learning/Clone/RAGBOOK/note/train_book/13374-xin-em-dung-dan-chut-tap-1-thuviensach.vn.pdf',
 '/home/skmlab1/data/data/QuangMinh/Learning/Clone/RAGBOOK/note/train_book/235-thuong-de-giang-chan-ly-thuviensach.vn.pdf',
 '/home/skmlab1/data/data/QuangMinh/Learning/Clone/RAGB

In [4]:
len(file_names)

10

In [5]:
'''
full_text = ""
for file_path in file_names:
    full_text += read_pdf(file_path) + "\n"
    
with open("/home/skmlab1/data/data/QuangMinh/Learning/Clone/RAGBOOK/note/text/full_text.txt", 'w') as file:
    file.write(full_text)
'''

'\nfull_text = ""\nfor file_path in file_names:\n    full_text += read_pdf(file_path) + "\n"\n    \nwith open("/home/skmlab1/data/data/QuangMinh/Learning/Clone/RAGBOOK/note/text/full_text.txt", \'w\') as file:\n    file.write(full_text)\n'

In [6]:
title_names = []
for filename in os.listdir(folder_path):
    title_names.append(filename.rstrip(".pdf"))

In [7]:
title_names

['.ipynb_checkpoints',
 '10550-loan-the-anh-hung-tap-1-thuviensach.vn',
 '11441-nhung-chang-trai-huyen-thoai-tap-1-thuviensach.vn',
 '12844-bi-mat-cua-nhung-bi-mat-tap-1-bai-noi-ve-bi-mat-cua-hoa-vang-thuviensach.vn',
 '13270-vong-tron-da-thieng-tap-1-thuviensach.vn',
 '13354-world-of-warcraft-chronicle-quyen-1-thuviensach.vn',
 '13374-xin-em-dung-dan-chut-tap-1-thuviensach.vn',
 '235-thuong-de-giang-chan-ly-thuviensach.vn',
 '5006-nep-cu-tin-nguong-viet-nam-quyen-ha-thuviensach.vn',
 '5077-nghin-xua-van-hien-tap-1-thuviensach.vn',
 '8518-hon-tuyet-lan-cuoc-doi-va-su-nghiep-cua-warren-buffett-tap-1-thuviensach.vn']

In [8]:
'''
path = "/home/skmlab1/data/data/QuangMinh/Learning/Clone/RAGBOOK/note/text/"
for i in range(len(title_names)):
    full_text = read_pdf(file_names[i])
    with open(path + title_names[i] + ".txt", 'w') as file:
        file.write(full_text)
'''

'\npath = "/home/skmlab1/data/data/QuangMinh/Learning/Clone/RAGBOOK/note/text/"\nfor i in range(len(title_names)):\n    full_text = read_pdf(file_names[i])\n    with open(path + title_names[i] + ".txt", \'w\') as file:\n        file.write(full_text)\n'

In [9]:
len(title_names)

11

In [10]:
title_names = title_names[1:]

In [11]:
len(title_names)

10

In [12]:
#path = "/home/skmlab1/data/data/QuangMinh/Learning/Clone/RAGBOOK/note/text/"
doc_text = []
for i in range(len(title_names)):
    full_text = read_pdf(file_names[i])
    doc_text.append(full_text)

In [13]:
len(doc_text)

10

In [14]:
'''
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_text(doc_text)
'''

'\ntext_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)\ndocs = text_splitter.split_text(doc_text)\n'

In [15]:
#text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=0)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=0)

In [16]:
titles = []
texts = []
for i in range(len(doc_text)):
    chunk = text_splitter.split_text(doc_text[i])
    num = len(chunk)
    #print(num)
    for k in range(num):
        titles.append(title_names[i])
        texts.append(chunk[k] + "\t" + title_names[i])

In [17]:
len(texts)

6637

In [18]:
'''
documents = []
for i in range(len(file_names)):
    loader = PyPDFLoader(file_names[i])
    documents.extend(loader.load())
'''

'\ndocuments = []\nfor i in range(len(file_names)):\n    loader = PyPDFLoader(file_names[i])\n    documents.extend(loader.load())\n'

In [19]:
'''
with open('/home/skmlab1/data/data/QuangMinh/Learning/Clone/RAGBOOK/note/text/chunk.txt', 'w') as file:
    for i in range(len(texts)):
        file.write(texts[i] + "<endofchunk>")
'''

'\nwith open(\'/home/skmlab1/data/data/QuangMinh/Learning/Clone/RAGBOOK/note/text/chunk.txt\', \'w\') as file:\n    for i in range(len(texts)):\n        file.write(texts[i] + "<endofchunk>")\n'

In [20]:
# all-mpnet-base-v2
# vietnamese-sbert
# BAAI/bge-m3
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
#embedding_function = SentenceTransformerEmbeddings(model_name="BAAI/bge-m3")

In [21]:
embedding_function

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [22]:
# load it into Chroma
#db = Chroma.from_documents(documents, embedding_function, persist_directory="/home/skmlab1/data/data/QuangMinh/Learning/Clone/RAGBOOK/note/chroma_db")
#db = Chroma.from_texts(texts, embedding_function, persist_directory="/home/skmlab1/data/data/QuangMinh/Learning/Clone/RAGBOOK/note/chroma_db")
db = Chroma.from_texts(texts, embedding_function, persist_directory="/home/skmlab1/data/data/QuangMinh/Learning/Clone/RAGBOOK/note/db/chroma_db_ver1")
#db = Chroma.from_texts(texts, embedding_function, persist_directory="/home/skmlab1/data/data/QuangMinh/Learning/Clone/RAGBOOK/note/db/chroma_db_bge")

In [23]:
db